In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

In [ ]:
# 데이터에 Null 값 있는지 확인해보기
train_df.isnull().sum().any()

In [ ]:
# X와 y(target) 분리
train_X = train_df.drop(['ID_code','target'], axis = 1)
train_y = train_df['target']

In [ ]:
# target 데이터가 편향적인지 확인해보기
train_y.value_counts()

In [ ]:
graph_1 = sns.countplot(train_y)

#### <그래프 1>: 클래스 간의 비율이 상당히 차이가 나므로, 매우 많이 비대칭적

#### Q1. 그렇다면 학습이 '0'에 치우치게 될텐데.. 데이터가 20만개라는 점과 imbalance 데이터라는 점에서 Resampling의 under-sampling을 통해 해결해보자

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
 
rus = RandomUnderSampler(random_state=0)
rus.fit(train_X, train_y)
resample_X, resample_y = rus.fit_resample(train_X, train_y)

In [ ]:
resample_X

In [ ]:
graph_2 = sns.countplot(resample_y)

#### <그래프2> : target 변수가 동일한 비율로 resampling 되었을 뿐만 아니라, train_데이터의 갯수 역시 줄어듦

In [ ]:
X = pd.DataFrame(resample_X)
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
X.columns = features
y = pd.DataFrame(resample_y, columns=['target'])

In [ ]:
# 의사결정나무의 변수의 중요도를 통해 200개의 변수의 중요도에 대해 파악해보자
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=5, random_state=2)
forest = forest.fit(X,y)
forest.feature_importances_

In [ ]:
feature_list = pd.DataFrame(forest.feature_importances_)
feature_list.columns = ['importance']
feature_list.sort_values('importance', ascending = False).head()

In [ ]:
# 해당모델에서 중요하게 생각되는 변수들을 선택해보자
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(forest, prefit=True)
X = sfm.transform(X)

#### Q2. 변수를 더 줄일 수 있는 방법은? 다중공선성을 이용해 변수를 줄여보자

In [ ]:
X = pd.DataFrame(X)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VLF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif.sort_values(["VLF Factor"], ascending=[False])

In [ ]:
X.drop([33,4,27], axis=1, inplace=True)

In [ ]:
df = pd.concat([X,y], axis=1)

In [ ]:
# 층화추출하는 함수 
def sampling_func(data, sample_pct):
    np.random.seed(123)
    N = len(data)
    sample_n = int(len(data)*sample_pct)
    sample = data.take(np.random.permutation(N)[:sample_n])
    return sample

In [ ]:
sample_set = df.groupby('target',group_keys=False).apply(sampling_func, sample_pct=0.05)
len(sample_set)

In [ ]:
X = sample_set.iloc[:,:-1]
y = sample_set.iloc[:,-1::]

In [ ]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
scaled_X = sc_X.fit_transform(X)
warnings.filterwarnings(action='once')

In [ ]:
# SVM - linear
from sklearn.svm import SVC

svm_linear = SVC(kernel='linear', C=1.0, random_state=0)
svm_linear.fit(scaled_X, y)
svm_linear.score(scaled_X, y)

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(svm_linear, scaled_X, y, cv=3).mean()

In [ ]:
# SVM - rbf
svm_rbf = SVC(kernel='rbf', gamma=0.0001, random_state=0)
svm_rbf.fit(scaled_X, y)
svm_rbf.score(scaled_X, y)
cross_val_score(svm_rbf, scaled_X, y,cv=3).mean()

In [ ]:
# poly
svm_poly = SVC(kernel='poly', C=1.0, random_state=0)
svm_poly.fit(scaled_X, y)
svm_poly.score(scaled_X, y)
cross_val_score(svm_poly, scaled_X, y, cv=3).mean()

#### Q3 낮은 값이 나왔는데, 이것은 parameters의 탓일까? feature selection의 탓일까? 그렇다면 우선, 최적의 parameters를 추정해보자

In [ ]:
# linear일 경우
from sklearn.svm import SVC

svm_linear = SVC(kernel='linear', C=1.0, random_state=0)
svm_linear.fit(scaled_X, y)

In [ ]:
param_grid = {
    'C': [i for i in range(1,25)],
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=svm_linear, param_grid=param_grid, cv=3, n_jobs=-1)
grid = grid.fit(scaled_X, y)
grid.best_params_
grid.best_score_

In [ ]:
# rbf일 경우
from sklearn.svm import SVC

svm_rbf = SVC(kernel='rbf', C=1.0, random_state=0)
svm_rbf.fit(scaled_X, y)

In [ ]:
param_grid = {
    'C': [i for i in range(1,25)],
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=svm_rbf, param_grid=param_grid, cv=3, n_jobs=-1)
grid = grid.fit(scaled_X, y)
grid.best_params_

In [ ]:
param_grid = {
    'C' : [1],
    'gamma': [0.0001,0.001,0.01,0.1,1,10,100],
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=svm_rbf, param_grid=param_grid, cv=3, n_jobs=-1)
grid = grid.fit(scaled_X, y)
grid.best_params_
grid.best_score_

In [ ]:
# poly의 경우
from sklearn.svm import SVC

svm_poly = SVC(kernel='poly', C=1.0, random_state=0)
svm_poly.fit(scaled_X, y)

In [ ]:
param_grid = {
    'C': [i for i in range(1,25)],
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=svm_poly, param_grid=param_grid, cv=3, n_jobs=-1)
grid = grid.fit(scaled_X, y)
grid.best_params_

In [ ]:
param_grid = {
    'C' : [1],
    'gamma': [0.0001,0.001,0.01,0.1,1,10,100],
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=svm_poly, param_grid=param_grid, cv=3, n_jobs=-1)
grid = grid.fit(scaled_X, y)
grid.best_params_
grid.best_score_

#### Q4. 전체적으로 gridSearch를 통해 얻어진 best_score_가 낮게 나온다. 왜그럴까? 아무래도 resample(undersampling)을 통해서 재구성된 데이터가 1과 0을 찾을 때 0에 대한 학습이 기존의 그것보다 못해서 그런 것 같다. 그렇다면 test_df의 실제 target값이 0이 많다면 안좋은 결과를 1이 많다면 조금 더 나은 결과를 얻을 수 있겠다!

## Test 데이터 예측: 결과가 안 좋았으니까, 차원을 축소시키지 않은 데이터(학습을 줄인 데이터)에서 결과를 도출

In [ ]:
# 1) 데이터셋 준비
X = pd.DataFrame(resample_X)
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
X.columns = features
y = pd.DataFrame(resample_y, columns=['target'])

In [ ]:
real_df = pd.concat([X,y], axis=1)
sample_set = real_df.groupby('target',group_keys=False).apply(sampling_func, sample_pct=0.05)
X = sample_set.iloc[:,:-1]
y = sample_set.iloc[:,-1::]
scaled_X = sc_X.fit_transform(X)
warnings.filterwarnings(action='once')

In [ ]:
# 2) 테스트 데이터 스케일링
test_df.drop('ID_code', axis = 1, inplace = True)
scaled_test = sc_X.fit_transform(test_df)
warnings.filterwarnings(action='once')

In [ ]:
# 3) 모델링
from sklearn.svm import SVC

svm_real = SVC(kernel='rbf', C=1.0, random_state=0)
svm_real.fit(scaled_X, y)
y_pred = svm_real.predict(scaled_test)

In [ ]:
# 4) 제출
test_df = pd.read_csv("test.csv")
submission_data= test_df["ID_code"]
submission_data = pd.concat([submission_data, pd.Series(y_pred)], axis= 1)
submission_data.columns = ['ID_code', 'target']

In [ ]:
submission_data['target'].value_counts()

In [ ]:
submission_data.to_csv(r"C:\Users\ehj14\Desktop\TOBIGS\5week\SVM\submission_data_Ver2.csv", header=True, index=False)